In [1]:
import pymongo
import numpy as np
import pandas as pd
import datetime,time

In [5]:
# Connect with mongodb

try:
    with open("credentials.txt", 'r', encoding='utf-8') as f:
        [name,password,url,dbname]=f.read().splitlines()
        conn=pymongo.MongoClient("mongodb://{}:{}@{}/{}".format(name,password,url,dbname))
    print ("Connected successfully!!!")
except pymongo.errors.ConnectionFailure as e:
    print ("Could not connect to MongoDB: %s" % e) 
    

    
db = conn['tweempact']
collection = db['tweets']

Connected successfully!!!


In [6]:
db.collection_names()

C:\Users\Albert Prat\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  """Entry point for launching an IPython kernel.


['system.indexes', 'tweets_noRT', 'users_noRT', 'tweets', 'tweet_users']

In [7]:
def read_mongo(db, collection, query={}, host='localhost', port=27017, username=None, password=None, no_id=True):
    """ Read from Mongo and Store into DataFrame """
    
    # Make a query to the specific DB and Collection
    cursor = db[collection].find(query)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if no_id:
        del df['_id']
    return df

In [19]:
df1 = read_mongo(db, "tweets")
df2 = read_mongo(db, "tweets_noRT")

In [20]:
print(len(df1), len(df2))

10490 19119


In [13]:
 def author_dataset_conversion(df):   
    import time
    import numpy as np
    import pandas as pd

    cleaned = df[["id", "created_at","entities", "extended_entities","favorite_count","retweet_count","source","text", "truncated","user","quoted_status_id"]]
    ids = [user["id"] for user in cleaned["user"]]
    cleaned["id_num"] = ids

    #preparing output variables
    output = pd.DataFrame()
    users = []
    mean_RT_last10,mean_FC_last10 = [],[]
    sd_RT,sd_FC  = [],[]
    y1,y2=[],[]

    for _id in np.unique(ids):
        subset = cleaned[cleaned["id_num"]==_id].reset_index(drop=True)

        #filtering tweets from all tweets and RT
        l = list(subset["text"])
        filtr = ["RT" not in i for i in l]
        subset = subset[pd.Series(filtr)]
        if len(subset)>=11:
            reindexed = subset["created_at"].reset_index(drop=True)
            #convert tweet date into python date
            subset["created_at_pydate"] = [time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(reindexed[i],'%a %b %d %H:%M:%S +0000 %Y'))for i in range(0,len(reindexed))]
            subset = subset.sort_values(by="created_at_pydate").reset_index(drop=True)

            last10_tweets = subset[-11:-1]
            last_tweet = subset[-1:]
            #RT parameters last 10 tweets
            mean_RT_last10.append(np.mean(last10_tweets["retweet_count"]))
            sd_RT.append(np.std(last10_tweets["retweet_count"]))
            #FC parameters last 10 tweets
            mean_FC_last10.append(np.mean(last10_tweets["favorite_count"]))
            sd_FC.append(np.std(last10_tweets["favorite_count"]))
            #take json user
            j_user = subset[-1:]["user"].reset_index(drop=True)[0]
            users.append(j_user)
            #parameters of last tweet
            y1.append(np.mean(last_tweet["favorite_count"]))
            y2.append(np.mean(last_tweet["retweet_count"]))
    output["j_user"]=users
    output["RT_l10"]=mean_RT_last10
    output["sd_RT"]=sd_RT
    output["FC_l10"]=mean_FC_last10
    output["sd_FC"]=sd_FC
    output["FC"]=y1
    output["RT"]=y2
    return output

In [22]:
output1 = author_dataset_conversion(df1)
output2 = author_dataset_conversion(df2)

C:\Users\Albert Prat\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [26]:
print(len(output1),len(output2))

186 1406


In [33]:
output = pd.concat([output1, output2], axis = 0)

In [35]:
pd.DataFrame.to_csv(output, "C:/Users/Albert Prat/Documents/tweempact/datasets/author.csv")

In [54]:
def text_dataset(df):
    import re
    import pandas as pd
    l = list(df["text"])
    filtr = ["RT" not in i for i in l]
    df = df[filtr].reset_index(drop=True)
    
    Tweet =[re.sub("[(\n)(\r)]"," ",tweet) for tweet in df["text"]]
    entities = [tweet for tweet in df["entities"]]
    FC = [tweet for tweet in df["favorite_count"]]
    RT = [tweet for tweet in df["retweet_count"]]
    id_ = [tweet for tweet in df["id"]]
    j_user = [tweet for tweet in df["user"]]
    followers = [tweet["followers_count"] for tweet in df["user"]]
    
    CA = df["created_at"]
    CA_l = [time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(CA[i],'%a %b %d %H:%M:%S +0000 %Y'))for i in range(0,len(CA))]
    
    output = pd.DataFrame([id_,Tweet,j_user,CA_l,entities,followers, FC,RT]).T
    output.reset_index(drop=True)
    output.columns = ["id", "Tweet", "j_user","created_at", "entities", "followers_user", "FC","RT"]
    return output

In [55]:
output1 = text_dataset(df1)
output2 = text_dataset(df2)

In [62]:
#pd.DataFrame.to_csv(output2, "C:/Users/Albert Prat/Documents/tweempact/datasets/tweet.csv")

In [140]:
def trending_topic_df(df, path="c:/Users/Albert Prat/chromedriver.exe", just_dates=False):    
    from selenium import webdriver
    import pandas as pd
    import numpy as np
    import datetime
    
    if just_dates==False:
    # preparing elements to iterate
        textdf = df["Tweet"]
        cadf = df["created_at"]
        cadf = [date[:10] for date in cadf]
        cadf = np.unique(cadf)
        cadf = [date.split("-") for date in cadf]
        iterable_for_adding_dummy = [date[2]+date[1]+date[0] for date in cadf]
        # to be able to work with <, >
    if just_dates == True:
        iterable_for_adding_dummy = df
    # setting the minimum date to look for and preparing the filters   
    u = [datetime.datetime.strptime(date, "%d%m%Y") for date in iterable_for_adding_dummy]
    threshold = datetime.datetime.strptime("01032015", "%d%m%Y")
    filtr = [u[i]>=threshold for i in range(len(u))]
    filtr1 = [u[i]<threshold for i in range(len(u))]
    
    iterable = np.array(iterable_for_adding_dummy)[filtr]

    # which of the unique dates is before threshold?
    indices = [i for i, x in enumerate(filtr1) if x == True]
    
    # entering to browser and select the date box to be inputted
    browser = webdriver.Chrome(path)
    browser.get("https://trendogate.com/")
    date_dd = browser.find_element_by_xpath("//*[@id='userheaderbox']/div/form[2]/div[2]/input")
    
    # preparing output lists
    error = []
    grepl = []
    out = []
    for i in range(len(iterable)):
        browser.find_element_by_xpath("//select[@name='place']/option[text()='Spain']").click()
        date_dd = browser.find_element_by_xpath("//*[@id='userheaderbox']/div/form[2]/div[2]/input")
        if just_dates == False:
            date = iterable[i][:10]
            date = date.split("-")
            date = date[2]+date[1]+date[0]
        if just_dates == True:
            date = iterable[i]
        date_dd.send_keys(date)
        browser.find_element_by_xpath("//*[@id='userheaderbox']/div/form[2]/button").click()

        if len(browser.find_elements_by_class_name("alert")) == 1:
            error.append(date)
            out.append(None)
            pass
        else:
            b = browser.find_element_by_class_name("col-lg-8")
            lgi = b.find_elements_by_class_name("list-group-item")
            TT = [el.text for el in lgi]
            TT = ",".join(TT)
            out.append(TT)
    for index in indices:
        out.insert(index, None)
    dates = [date[:2]+"/"+date[2:4]+"/"+date[4:] for date in iterable_for_adding_dummy]
    tocsv = pd.DataFrame([dates, out]).T
    tocsv.columns=["date", "trending_topics"]
    return tocsv

In [1051]:
tocsv = trending_topic_df(output2)

In [141]:
def widen_TT_csv(df):
    old = pd.read_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\tweet.csv")
    ca_old = old["created_at"]
    ca_old = [date[:10] for date in ca_old]
    ca_old = np.unique(ca_old)
    ca_old = [date.split("-") for date in ca_old]
    ca_old = [date[2]+date[1]+date[0] for date in ca_old]
    
    ca_new = df["created_at"]
    ca_new = [date[:10] for date in ca_new]
    ca_new = np.unique(ca_new)
    ca_new = [date.split("-") for date in ca_new]
    ca_new = [date[2]+date[1]+date[0] for date in ca_new]
    
    new_dates = set(ca_new) - set(ca_old)
    
    tocsv = trending_topic_df(list(new_dates), just_dates = True)
    return tocsv

In [142]:
trend = widen_TT_csv(output2)

In [149]:
old = pd.read_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\trending_topics.csv")

In [153]:
output = pd.concat([old, trend], axis = 0).reset_index(drop = True)

C:\Users\Albert Prat\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


In [172]:
output = output.drop(output.columns[0], axis = 1)

In [253]:
import re

In [270]:
np.where(trending_topics["date"]=="14/08/2013")

(array([0], dtype=int64),)

In [267]:
    ca = list(trending_topics["date"])
    unique_date = [date.split("/") for date in ca]
    unique_date = [date[2]+date[1]+date[0] for date in unique_date]
    unique_date = np.unique(unique_date)
    dates = []
    for date in unique_date:
        date_ = date[:4]+"/"+date[4:6]+"/"+date[6:]
        date_=date_.split("/")
        date_ = date_[2]+date_[1]+date_[0]
        dates.append(date_)


In [277]:
index = np.where(trending_topics["date"]==(sp_d[1][:2]+"/"+sp_d[1][2:4]+"/"+sp_d[1][4:]))

In [280]:
index[0][0]

399

In [294]:
import unidecode

In [311]:
output2["Tweet"][8]

'Aquesta gent corrupta del @PPopular sempre diu que la corrupció és cosa del passat, però el passat del PP sembla se… https://t.co/NGzsq4nHyy'

In [346]:
string = ["SergiRoberto es un patata", "sergi roberto es un patata", "sergi és un patata", "hola roberto"]

In [348]:
for s in string:
    s = s.lower()
    print(bool(re.search(r"\b"+str("sergi roberto")+r"\b"+r"|\b"+str("sergiroberto")+r"\b", s)))

True
True
False
False


In [349]:
r"\b"+str("sergi roberto")+r"\b"+r"|\b"+str("sergiroberto")+r"\b"

'\\bsergi roberto\\b|\\bsergiroberto\\b'

In [341]:
output.to_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\trending_topics.csv")

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\Albert Prat\\Documents\\tweempact\\datasets\\trending_topics.csv'

In [482]:
for dataframe in [df1,output2]:
    for column in dataframe.columns:
        dtype = type(dataframe[column][0])
        if dtype == dict:
            print("a")
        else:
            print(dtype)

<class 'numpy.int64'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.float64'>
<class 'str'>
<class 'str'>
a
<class 'str'>
a
<class 'int'>
<class 'int'>
<class 'int'>
<class 'str'>
<class 'numpy.float64'>


In [485]:
df1["j_user"][0]

"{'id': 2664945430, 'id_str': '2664945430', 'name': 'Fútbol Club Jumilla', 'screen_name': 'FCJumilla', 'location': 'Jumilla, España', 'description': 'Perfil Oficial del Fútbol Club Jumilla.\\nSegunda División B / Grupo IV @RFEF', 'url': 'https://t.co/VYPxc3mFj6', 'entities': {'url': {'urls': [{'url': 'https://t.co/VYPxc3mFj6', 'expanded_url': 'http://www.futbolclubjumilla.es', 'display_url': 'futbolclubjumilla.es', 'indices': [0, 23]}]}, 'description': {'urls': []}}, 'protected': False, 'followers_count': 7162, 'friends_count': 334, 'listed_count': 92, 'created_at': 'Wed Jul 02 19:09:45 +0000 2014', 'favourites_count': 2804, 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'verified': False, 'statuses_count': 8466, 'lang': 'es', 'contributors_enabled': False, 'is_translator': False, 'is_translation_enabled': False, 'profile_background_color': 'C0DEED', 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_image_url_https': 'h

In [479]:
df1 = df1.reset_index(drop = True)

In [463]:
df1 = pd.read_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\tweet.csv", index_col=0)

In [466]:
df1.columns

Index(['id', 'Tweet', 'j_user', 'created_at', 'entities', 'followers_user',
       'FC', 'RT', 'trending_topic'],
      dtype='object')

In [492]:
def append_dummy_TT(df):
    import re
    import unidecode
    import pandas as pd
    import ast
    
    textdf = df["Tweet"]
    trending_topics = pd.read_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\trending_topics.csv")
    TT = trending_topics["trending_topics"]
    indices = TT[TT.isnull()].index
    # get TT as a list of lists
    out = []
    for i in range(len(TT)):
        if i in indices:
            out.append(None)
        else:
            t = TT[i].split(",")
            out.append(t)
    # date management
    ca = list(df["created_at"])
    ca = [date[:10] for date in ca]
    sp_d = [date.split("-") for date in ca]
    sp_d = [date[2]+date[1]+date[0] for date in sp_d]
    # just keeping unique dates
    ca = list(trending_topics["date"])
    unique_date = np.unique(ca)
    unique_date = [date.split("/") for date in unique_date]
    unique_date = [date[2]+date[1]+date[0] for date in unique_date]
    # append sp_d for the sake of indexing
    df["spanish_date"]=sp_d
    grepl = []
    for i in range(len(sp_d)):
        ls = []
        date = df[df.index == i]["spanish_date"][i]
        tweet = textdf[i].lower()
        tweet = unidecode.unidecode(tweet)
        index = np.where(trending_topics["date"]==(sp_d[i][:2]+"/"+sp_d[i][2:4]+"/"+sp_d[i][4:]))
        TT = out[index[0][0]]
        if TT == None:
            grepl.append(None)
        else:
            for word in TT:
                word = word.lower()
                word = unidecode.unidecode(word)
                word1 = re.sub(" ", "", word)
                ls.append(bool(re.search(r"\b"+str(word)+r"\b"+r"|\b"+str(word1)+r"\b", tweet)))
            grepl.append(np.sum(ls))
    df = df.drop("spanish_date", axis = 1)
    df["trending_topic"]=grepl
    df.reset_index(drop = True)
    
    df1 = pd.read_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\tweet.csv", index_col=0)
    dropped = []
    for column in df.columns:
        dtype = type(df[column][0])
        if dtype == dict:
            df[column] = df[column].astype(str)
            dropped.append(column)
    df = pd.concat([df1,df], axis = 0).drop_duplicates().reset_index(drop=True)
    df.reset_index(drop = True)
    return df

In [493]:
out2 = append_dummy_TT(output2)

In [510]:
out2.to_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\tweet.csv")

In [511]:
out = pd.read_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\tweet.csv", index_col=0)

In [388]:
len(out2)

18968

In [367]:
out.to_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\tweet.csv")